In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

In [10]:
df_pois = pd.read_csv('Datasets/pois_filtrados.csv')
df_bus = pd.read_csv('Datasets/Paraderos.csv')
df_metro = pd.read_csv('Datasets/metro.csv')
df_deptos = pd.read_csv('Datasets/dataset_final.csv')

In [11]:
print(df_pois.info())
print(df_bus.info())
print(df_metro.info())
print(df_deptos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23145 entries, 0 to 23144
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   longitude  23145 non-null  float64
 1   latitude   23145 non-null  float64
 2   type       23145 non-null  object 
 3   class      23145 non-null  object 
 4   name       19691 non-null  object 
dtypes: float64(2), object(3)
memory usage: 904.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12635 entries, 0 to 12634
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         12635 non-null  int64  
 1   id          12635 non-null  int64  
 2   codigo      12635 non-null  object 
 3   simt        11774 non-null  object 
 4   tipo        12635 non-null  object 
 5   clasificac  12635 non-null  object 
 6   comuna      12635 non-null  object 
 7   zona        12635 non-null  object 
 8   eje         12635 non-null  o

In [14]:
# 📍 Definir el centro del mapa
centro_mapa = [-33.45, -70.66]

# 🗺️ Crear el mapa de Folium
m = folium.Map(location=centro_mapa, zoom_start=12)

# 🎨 Colores para cada tipo de elemento
colors = {
    'POIs': 'blue',
    'Bus': 'red',
    'Metro': 'lightgreen',
    'Departamentos': 'purple'
}

# 🔵 Añadir POIs al mapa (azul)
# Para POIs, las columnas son 'longitude' y 'latitude'
for idx, row in df_pois.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color=colors['POIs'],
        fill=True,
        fill_color=colors['POIs'],
        fill_opacity=0.7,
        tooltip=f"{row['type']} - {row['class']}"
    ).add_to(m)

# 🔴 Añadir paradas de bus al mapa (rojo)
# Para Bus, las columnas son 'longitud' y 'latitud'
for idx, row in df_bus.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=2,
        color=colors['Bus'],
        fill=True,
        fill_color=colors['Bus'],
        fill_opacity=0.7
    ).add_to(m)

# 🟢 Añadir estaciones de metro al mapa (verde)
# Para Metro, las columnas son 'longitud' y 'latitud'
for idx, row in df_metro.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=2,
        color=colors['Metro'],
        fill=True,
        fill_color=colors['Metro'],
        fill_opacity=0.7
    ).add_to(m)

# 🟣 Añadir departamentos al mapa (morado)
# Para Deptos, las columnas son 'longitud' y 'latitud'
for idx, row in df_deptos.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=2,
        color=colors['Departamentos'],
        fill=True,
        fill_color=colors['Departamentos'],
        fill_opacity=0.7
    ).add_to(m)

# 💾 Guardar el mapa como un archivo HTML
m.save("mapa_elementos_santiago.html")
print("Mapa guardado como mapa_elementos_santiago.html")

Mapa guardado como mapa_elementos_santiago.html


In [15]:
print(df_pois['class'].unique())
print('\n')
print(df_pois['type'].unique())

['sport_and_leisure' 'medical' 'educational' 'veterinary'
 'food_and_drink_stores' 'arts_and_entertainment' 'food_and_drink'
 'building' 'park_like' 'public_facilities' 'religion' 'education']


['Sports Centre' 'Hospital' 'Clinic' 'Kindergarten' 'Veterinary'
 'Supermarket' 'Arts Centre' 'Cafe' 'Restaurant' 'Dentist' 'Commercial'
 'Fast Food' 'Playground' 'Post Office' 'Park' 'Police' 'Bakery'
 'Convenience' 'School' 'Courthouse' 'Place Of Worship' 'Fitness Centre'
 'Pharmacy' 'Swimming Pool' 'Bar' 'Confectionery' 'Monastery'
 'Fire Station' 'Pastry' 'Yes' 'Library' 'Doctors' 'University' 'Pitch'
 'Alcohol' 'Greengrocer' 'Track' 'Nature Reserve' 'Attraction' 'Viewpoint'
 'Pub' 'Gallery' 'Ice Cream' 'Museum' 'College' 'Community Centre' 'Civic'
 'Theatre' 'Beverages' 'Marketplace' 'Butcher' 'Stadium'
 'Recreation Ground' 'Fitness Station' 'Social Facility' 'Industrial'
 'Allotments' 'Ruins' 'Townhall' 'Church' 'Seafood' 'Food Court' 'Cinema'
 'Nightclub' 'Office' 'Picnic Site' 'Construc

In [53]:
for class_val, group in df_pois.groupby('class'):
    type_counts = group['type'].value_counts()
    
    # 📝 Formatear la salida para incluir el conteo
    formatted_types = []
    for type_name, count in type_counts.items():
        formatted_types.append(f"{type_name} ({count})")
    
    print(f"Clase: {class_val}")
    print(f"  Tipos únicos: {', '.join(formatted_types)}")
    print("-" * 30)

Clase: arts_and_entertainment
  Tipos únicos: Museum (83), Theatre (71), Arts Centre (59), Gallery (47), Stadium (32), Cinema (32), Attraction (31), Track (19), Amusement Arcade (5), Water Park (4), Casino (4), Zoo (4), Theme Park (3), Aquarium (1), Raceway (1)
------------------------------
Clase: building
  Tipos únicos: Commercial (172), Industrial (86), Office (46), Church (41), Warehouse (26), Roof (20), Construction (16), Chapel (13), Civic (9), Grandstand (8), Transportation (7), Hut (6), Public (6), Ruins (6), Service (5), Hotel (4), Fire Station (3), Train Station (3), Greenhouse (1), Stage (1), Stadium (1), Sports Centre (1), Temple (1), Healthcare (1), Sala De Juegos (1), Hangar (1), Oficinas (1), Condominium (1), Salas Y Cacino (1), Administrative (1), Edificio Luis Nuñez (1), Government (1), Concierger (1), Dormitory (1)
------------------------------
Clase: education
  Tipos únicos: University (467), College (70)
------------------------------
Clase: educational
  Tipos ú

In [57]:
tipo_a_visualizar = 'Transportation' 
df_tipo_especifico = df_pois[df_pois['type'] == tipo_a_visualizar]
print(f"Filas con type = '{tipo_a_visualizar}':")
print(df_tipo_especifico.sample(7))
print(f"\nTotal de entradas para '{tipo_a_visualizar}': {len(df_tipo_especifico)}")

Filas con type = 'Transportation':
       longitude   latitude            type     class  \
6476  -70.652862 -33.437202  Transportation  building   
10509 -70.655597 -33.446183  Transportation  building   
5278  -70.606793 -33.462607  Transportation  building   
7805  -70.605397 -33.454534  Transportation  building   
2147  -70.658197 -33.444386  Transportation  building   
21303 -70.615828 -33.499331  Transportation  building   
21608 -70.729350 -33.365945  Transportation  building   

                                                    name  
6476                          Estación Plaza de Armas L3  
10509  Centro Integrado de Control del Metro de Santiago  
5278                              Metro Estadio Nacional  
7805       Centro de Control de Operaciones Líneas 3 y 6  
2147                        Terminal de Buses Los Héroes  
21303                                        San Joaquín  
21608                              Metro Plaza Quilicura  

Total de entradas para 'Transportat

In [58]:
tipo_a_eliminar = 'Transportation' 
print(f"Tamaño original de df_pois: {len(df_pois)} filas")
df_pois.drop(df_pois[df_pois['type'] == tipo_a_eliminar].index, inplace=True)
print(f"Tamaño de df_pois después de eliminar '{tipo_a_eliminar}': {len(df_pois)} filas")
print(df_pois.sample(10))

Tamaño original de df_pois: 21921 filas
Tamaño de df_pois después de eliminar 'Transportation': 21914 filas
       longitude   latitude         type                  class  \
22343 -70.632720 -33.567154        Pitch      sport_and_leisure   
10432 -70.520918 -33.352532       Bakery  food_and_drink_stores   
6048  -70.607331 -33.441331   University              education   
13816 -70.627437 -33.445915   Restaurant         food_and_drink   
10856 -70.662249 -33.445280   University              education   
19307 -70.704650 -33.471499   Playground              park_like   
2819  -70.672274 -33.428903  Marketplace  food_and_drink_stores   
15236 -70.705289 -33.598814   University              education   
8855  -70.808298 -33.569268         Park              park_like   
13236 -70.556305 -33.570420  Supermarket  food_and_drink_stores   

                                       name  
22343               Canchas Campus Antumapu  
10432              Chocolateria Varsovienne  
6048            

Se eliminaron los type Yes, Adult Gaming Centre, Events Venue, Community Centre, Post Office, Courthouse, Social Facility, Townhall, Public Building, Prison, Viewpoint, Transportation 